In [1]:
from __future__ import annotations
import time
import warnings
from pathlib import Path
from typing import Dict

import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.pipeline import Pipeline
from tqdm.auto import tqdm

In [2]:
tqdm.pandas()

### Configuration:

In [3]:
from sklearn.preprocessing import RobustScaler

from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings("ignore", category=UserWarning)

# ───────────────────────────── configuration ────────────────────────────── #
# List of trip IDs to drop from the dataset
DROP_TRIPS      = [10257]
# Base columns used for training the OC-SVM
BASE_COLUMNS    = [
    "speed_over_ground", "dv", "dcourse", "ddraft",
    "zone",
    "x_km", "y_km", "dist_to_ref", "route_dummy"
]

# Geographical zones defined by [lat_max, lat_min, lon_max, lon_min]
ZONES           = [[53.8, 53.5, 8.6, 8.14], [53.66, 53.0, 11.0, 9.5]]

# Radii (in km) defining "port" and "approach" zones (currently unused in logic)
R_PORT, R_APP   = 5.0, 15.0
# Earth radius in km, used for haversine distance calculation
EARTH_R         = 6_371.0

# Cross-validation settings
CV_FOLDS = 3
RANDOM_STATE = 42

### Functions for data preparation and feature engineering:

In [4]:
def haversine(lat1, lon1, lat2, lon2):
    """Vectorized haversine distance (km)."""
    lat1, lon1, lat2, lon2 = map(np.radians, (lat1, lon1, lat2, lon2))
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    return 2 * EARTH_R * np.arcsin(np.sqrt(a))

In [5]:
def load_and_prepare(path: str) -> pd.DataFrame:
    """
    Load parquet, drop specified trips, parse dates, compute delta-features,
    map labels and one-hot port zones.
    """
    df = pd.read_parquet(path, engine="pyarrow")
    print(f"Loaded {len(df):,} rows, dropping {len(df[df.trip_id.isin(DROP_TRIPS)]):,} rows from {DROP_TRIPS}")
    df = df[~df.trip_id.isin(DROP_TRIPS)].reset_index(drop=True)

    for col in ("start_time", "end_time", "time_stamp"):
        df[col] = pd.to_datetime(df[col])

    df = df.dropna(subset=["ship_type"]).reset_index(drop=True)
    df["y_true"]   = df["is_anomaly"].map({True: 1, False: 0})
    df["route_id"] = df["start_port"] # Using start_port as route_id for this pipeline

    # per-point deltas
    df = df.sort_values(["trip_id", "time_stamp"])
    df["dv"]      = df.groupby("trip_id")["speed_over_ground"].diff().abs().fillna(0)
    df["dcourse"] = df.groupby("trip_id")["course_over_ground"].diff().abs().fillna(0)
    df["ddraft"]  = df.groupby("trip_id")["draught"].diff().abs().fillna(0)

    # zones
    # port_coords is defined but not used in zone_label; it might be a remnant
    port_coords = (
        df.groupby("start_port")[["start_latitude", "start_longitude"]]
          .first()
          .to_dict("index")
    )

    def _in_any_rect(lat: float, lon: float) -> bool:
        """Checks if a given lat/lon is within any of the defined rectangular zones."""
        for lat_max, lat_min, lon_max, lon_min in ZONES:
            if lat_min <= lat <= lat_max and lon_min <= lon <= lon_max:
                return True
        return False

    def zone_label(row) -> int:
        """Assigns a zone label (0 for within zone, 1 for outside)."""
        if _in_any_rect(row.latitude, row.longitude):
            return 0 # In a defined zone
        return 1 # Outside all defined zones

    df["zone"] = df.progress_apply(zone_label, axis=1)
    # One-hot encode the 'zone' column. Note: this line is creating 'zone_0' and 'zone_1'
    # but only 'zone' (the numerical label) is used in BASE_COLUMNS.
    df = pd.concat([df, pd.get_dummies(df["zone"], prefix="zone")], axis=1)
    return df

In [6]:
def compute_average_route(df_route: pd.DataFrame, n_points: int = 100) -> np.ndarray:
    """
    Compute average trajectory for a route by resampling each trip to n_points
    along cumulative distance fraction, then averaging.
    """
    segments = []
    for _, trip in df_route.groupby("trip_id"):
        trip = trip.sort_values("time_stamp")
        lat, lon = trip.latitude.to_numpy(), trip.longitude.to_numpy()
        d = haversine(lat[1:], lon[1:], lat[:-1], lon[:-1])
        cum = np.concatenate(([0], np.cumsum(d)))
        if cum[-1] <= 0: # Handle cases with no movement within the trip
            continue
        frac   = cum / cum[-1]
        target = np.linspace(0, 1, n_points)
        segments.append(np.vstack([np.interp(target, frac, lat),
                                   np.interp(target, frac, lon)]).T)
    if not segments: # If no valid segments were found for the route
        return np.array([]) # Return an empty array or handle as error
    return np.mean(np.stack(segments, axis=0), axis=0)

In [7]:
def add_route_specific_features(df: pd.DataFrame, route: str) -> pd.DataFrame:
    """
    For a single route:
    • project lat/lon to local x_km, y_km
    • compute distance to average route (dist_to_ref)
    • add constant route_dummy = 1
    """
    df_r = df[df.route_id == route].copy()

    # local projection
    lat0, lon0 = df_r.latitude.mean(), df_r.longitude.mean()
    kx = 111.320 * np.cos(np.deg2rad(lat0)) # Conversion factor for longitude to km
    ky = 110.574 # Conversion factor for latitude to km
    df_r["x_km"] = (df_r.longitude - lon0) * kx
    df_r["y_km"] = (df_r.latitude  - lat0) * ky

    # distance to average trajectory
    avg = compute_average_route(df_r)
    if avg.size == 0: # Handle case where average route couldn't be computed
        df_r["dist_to_ref"] = 0.0 # Or fill with NaN, depending on desired behavior
        df_r["route_dummy"] = 1.0
        return df_r

    idx_map = df_r.index
    frac = np.zeros(len(df_r))
    for _, trip in tqdm(df_r.groupby("trip_id"), desc=f"Processing trips for route {route}"):
        pos = idx_map.get_indexer(trip.index)
        lat, lon = trip.latitude.values, trip.longitude.values
        d = haversine(lat[1:], lon[1:], lat[:-1], lon[:-1])
        cum = np.concatenate(([0], np.cumsum(d)))
        total = cum[-1] if cum[-1] > 0 else 1
        frac[pos] = cum / total

    # Ensure index for avg is within bounds (0 to 99 for 100 points)
    df_r["dist_to_ref"] = [
        haversine(lat, lon, avg[int(f * 99), 0], avg[int(f * 99), 1])
        for lat, lon, f in zip(df_r.latitude, df_r.longitude, frac)
    ]
    df_r["route_dummy"] = 1.0
    return df_r

## Training and Evaluation:

### Fucntions for model evaluation & cross-validation:

In [8]:
from sklearn.metrics import auc, precision_recall_curve


def evaluate_model(pipeline, X_train, X_test, y_test, nu, tau) -> Dict[str, float]:
    """Comprehensive model evaluation with multiple metrics."""
    scores_test = -pipeline.decision_function(X_test)
    preds = (scores_test > tau).astype(int)

    # Basic metrics
    if len(np.unique(y_test)) > 1:
        auc_roc = roc_auc_score(y_test, scores_test)
        precision, recall, _ = precision_recall_curve(y_test, scores_test)
        auc_pr = auc(recall, precision)
    else:
        auc_roc = auc_pr = 0.0

    # Confusion matrix metrics
    tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel() if len(np.unique(y_test)) > 1 else (len(y_test), 0, 0, 0)

    metrics = {
        'auc_roc': auc_roc,
        'auc_pr': auc_pr,
        'precision': tp / (tp + fp) if (tp + fp) > 0 else 0.0,
        'recall': tp / (tp + fn) if (tp + fn) > 0 else 0.0,
        'f1': 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0.0,
        'specificity': tn / (tn + fp) if (tn + fp) > 0 else 0.0,
        'balanced_accuracy': 0.5 * (tp / (tp + fn) + tn / (tn + fp)) if (tp + fn) > 0 and (tn + fp) > 0 else 0.0
    }

    return metrics

In [9]:
def cross_validate_ocsvm(X_norm, X_test, y_test, params, cv_folds=3):
    """Cross-validation for One-Class SVM parameter selection."""
    # Create train/validation splits from normal data
    n_samples = len(X_norm)
    indices = np.arange(n_samples)

    fold_size = n_samples // cv_folds
    cv_scores = []

    for fold in range(cv_folds):
        # Create validation split
        val_start = fold * fold_size
        val_end = (fold + 1) * fold_size if fold < cv_folds - 1 else n_samples

        val_indices = indices[val_start:val_end]
        train_indices = np.concatenate([indices[:val_start], indices[val_end:]])

        X_train_fold = X_norm[train_indices]
        X_val_fold = X_norm[val_indices]

        # Train model
        pipeline = Pipeline([
            ("scaler", params['scaler']),
            ("ocsvm", OneClassSVM(
                kernel=params['kernel'],
                gamma=params['gamma'],
                nu=params['nu'],
                verbose=False,
                shrinking=False
            ))
        ])

        pipeline.fit(X_train_fold)

        # Compute threshold on training data
        scores_train = -pipeline.decision_function(X_train_fold)
        tau = np.percentile(scores_train, 100 * (1 - params['nu']))

        # Evaluate on validation set (treating as potential anomalies)
        scores_val = -pipeline.decision_function(X_val_fold)
        # For validation, we expect most points to be normal, so we use a simple outlier score
        val_score = np.mean(scores_val < tau)  # Fraction classified as normal

        cv_scores.append(val_score)

    return np.mean(cv_scores)

In [10]:
import itertools

def comprehensive_parameter_search(X_norm, X_test, y_test, grid_name='focused_rbf', use_cv=True):
    """
    Comprehensive parameter search for One-Class SVM.

    Args:
        X_norm: Normal training data
        X_test: Test data (mixed normal/anomaly)
        y_test: Test labels
        grid_name: Which parameter grid to use
        use_cv: Whether to use cross-validation for parameter selection
    """
    param_grid = PARAMETER_GRIDS[grid_name]

    # Generate all parameter combinations
    param_names = list(param_grid.keys())
    param_values = list(param_grid.values())
    param_combinations = list(itertools.product(*param_values))

    print(f"Searching {len(param_combinations)} parameter combinations...")

    results = []
    best_score = -np.inf
    best_params = None
    best_pipeline = None
    best_tau = None

    with tqdm(param_combinations, desc="Parameter search") as pbar:
        for params in pbar:
            param_dict = dict(zip(param_names, params))

            try:
                # Create pipeline
                pipeline = Pipeline([
                    ("scaler", param_dict['scaler']),
                    ("ocsvm", OneClassSVM(
                        kernel=param_dict['kernel'],
                        gamma=param_dict['gamma'],
                        nu=param_dict['nu'],
                        verbose=False,
                        shrinking=False
                    ))
                ])

                # Fit on normal data
                pipeline.fit(X_norm)

                # Compute threshold
                scores_train = -pipeline.decision_function(X_norm)
                tau = np.percentile(scores_train, 100 * (1 - param_dict['nu']))

                # Evaluate
                if use_cv:
                    # Use cross-validation score as primary metric
                    cv_score = cross_validate_ocsvm(X_norm, X_test, y_test, param_dict, CV_FOLDS)
                    primary_score = cv_score
                else:
                    # Use test AUC as primary metric
                    metrics = evaluate_model(pipeline, X_norm, X_test, y_test, param_dict['nu'], tau)
                    primary_score = metrics['auc_roc']

                # Full evaluation on test set
                test_metrics = evaluate_model(pipeline, X_norm, X_test, y_test, param_dict['nu'], tau)

                result = {
                    'params': param_dict.copy(),
                    'primary_score': primary_score,
                    'tau': tau,
                    **test_metrics
                }
                results.append(result)

                # Update best
                if primary_score > best_score:
                    best_score = primary_score
                    best_params = param_dict.copy()
                    best_pipeline = pipeline
                    best_tau = tau

                # Update progress bar
                pbar.set_postfix({
                    'best_score': f'{best_score:.3f}',
                    'current': f'{primary_score:.3f}'
                })

            except Exception as e:
                print(f"Error with params {param_dict}: {e}")
                continue

    # Sort results by primary score
    results.sort(key=lambda x: x['primary_score'], reverse=True)

    return {
        'best_pipeline': best_pipeline,
        'best_params': best_params,
        'best_tau': best_tau,
        'best_score': best_score,
        'all_results': results
    }

In [11]:
def print_search_results(search_results, top_k=5):
    """Print formatted results from parameter search."""
    results = search_results['all_results']

    print(f"\n{'='*80}")
    print(f"PARAMETER SEARCH RESULTS - TOP {top_k}")
    print(f"{'='*80}")

    for i, result in enumerate(results[:top_k]):
        print(f"\nRank {i+1}:")
        print(f"  Primary Score: {result['primary_score']:.4f}")
        print(f"  AUC-ROC: {result['auc_roc']:.4f}")
        print(f"  AUC-PR: {result['auc_pr']:.4f}")
        print(f"  F1: {result['f1']:.4f}")
        print(f"  Balanced Accuracy: {result['balanced_accuracy']:.4f}")
        print(f"  Threshold (τ): {result['tau']:.4f}")

        params = result['params']
        print(f"  Parameters:")
        print(f"    nu: {params['nu']}")
        print(f"    gamma: {params['gamma']}")
        print(f"    kernel: {params['kernel']}")
        print(f"    scaler: {type(params['scaler']).__name__}")

## Start the training and evaluation process:

#### Configure parameter grids for search:

In [12]:
PARAMETER_GRIDS = {
    'comprehensive': {
        'nu': [0.001, 0.005, 0.01, 0.02, 0.03, 0.05, 0.1, 0.15, 0.2],
        'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1.0, 10.0],
        'kernel': ['rbf', 'poly', 'sigmoid'],
        'scaler': [StandardScaler(), RobustScaler(), MinMaxScaler()]
    },
    'focused_rbf': {
        'nu': [0.001, 0.005, 0.01, 0.02, 0.03, 0.05, 0.1],
        'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1.0],
        'kernel': ['rbf'],
        'scaler': [StandardScaler(), RobustScaler()]
    },
    'quick': {
        'nu': [0.01, 0.03],
        'gamma': ['scale'],
        'kernel': ['rbf'],
        'scaler': [StandardScaler()]
    }
}
# Fraction of normal points to include in the test set
TEST_FRACTION_N = 0.10

In [15]:
out_dir: str = "models_per_route"
df = load_and_prepare("all_trips_saved.parquet")

Loaded 912,566 rows, dropping 577 rows from [10257]


  0%|          | 0/911621 [00:00<?, ?it/s]

In [16]:
route = df.route_id.unique()[0]
# route = df[df.route_id == df.route_id.unique()[1]]
grid_name='quick'
route

'KIEL'

#### Start training for the specified route:

In [17]:
Path(out_dir).mkdir(exist_ok=True)

print(f"\n=== Training route: {route} with {grid_name} search ===")
t0 = time.time()

# Prepare route data
fr = add_route_specific_features(df, route)
X_norm = fr[fr.y_true == 0][BASE_COLUMNS].fillna(0).values
X_anom = fr[fr.y_true == 1][BASE_COLUMNS].fillna(0).values

if len(X_norm) == 0:
    print("  * No normal points, skipping this route.")

# Prepare test set
idx_anom = fr[fr.y_true == 1].index.to_numpy()
n_norm_test = max(1, int(TEST_FRACTION_N * len(X_norm)))
idx_norm = fr[fr.y_true == 0].sample(n=n_norm_test, random_state=RANDOM_STATE).index.to_numpy()

X_test = np.vstack([
    fr.loc[idx_anom, BASE_COLUMNS].fillna(0).values,
    fr.loc[idx_norm, BASE_COLUMNS].fillna(0).values
])
y_test = np.concatenate([
    np.ones(len(idx_anom), dtype=int),
    np.zeros(len(idx_norm), dtype=int)
])


=== Training route: KIEL with quick search ===


Processing trips for route KIEL:   0%|          | 0/420 [00:00<?, ?it/s]

In [18]:
search_results = comprehensive_parameter_search(X_norm, X_test, y_test, grid_name)
print_search_results(search_results)

Searching 2 parameter combinations...


Parameter search:   0%|          | 0/2 [00:00<?, ?it/s]


PARAMETER SEARCH RESULTS - TOP 5

Rank 1:
  Primary Score: 0.9646
  AUC-ROC: 0.9874
  AUC-PR: 0.9597
  F1: 0.8514
  Balanced Accuracy: 0.9655
  Threshold (τ): 0.0000
  Parameters:
    nu: 0.01
    gamma: scale
    kernel: rbf
    scaler: StandardScaler

Rank 2:
  Primary Score: 0.9443
  AUC-ROC: 0.9905
  AUC-PR: 0.9667
  F1: 0.7652
  Balanced Accuracy: 0.9694
  Threshold (τ): 0.0000
  Parameters:
    nu: 0.03
    gamma: scale
    kernel: rbf
    scaler: StandardScaler


### Final evaluation and model saving:

In [19]:
# Final evaluation
best_pipeline = search_results['best_pipeline']
best_tau = search_results['best_tau']
best_params = search_results['best_params']

scores_test = -best_pipeline.decision_function(X_test)
preds = (scores_test > best_tau).astype(int)

print(f"\n=== FINAL EVALUATION ===")
print(f"Best parameters: {best_params}")
print(f"Confusion Matrix:")
print(confusion_matrix(y_test, preds))
print(f"Classification Report:")
print(classification_report(y_test, preds, digits=3))
print(f"Route {route} completed in {time.time() - t0:.1f}s")


=== FINAL EVALUATION ===
Best parameters: {'nu': 0.01, 'gamma': 'scale', 'kernel': 'rbf', 'scaler': StandardScaler()}
Confusion Matrix:
[[7818  164]
 [  28  550]]
Classification Report:
              precision    recall  f1-score   support

           0      0.996     0.979     0.988      7982
           1      0.770     0.952     0.851       578

    accuracy                          0.978      8560
   macro avg      0.883     0.966     0.920      8560
weighted avg      0.981     0.978     0.979      8560

Route KIEL completed in 122.0s


In [20]:

# Save model
model_path = Path(out_dir) / f"ocsvm_{route}.pkl"
joblib.dump({
    "pipeline": best_pipeline,
    "features": BASE_COLUMNS,
    "tau": best_tau,
    "params": best_params,
    "search_results": search_results
}, model_path)

['models_per_route/ocsvm_KIEL.pkl']